## Dogs v Cats

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
#import skimage

In [2]:
PATH = "/home/embsysa/fastai/pytorch_classifiers-master/data/train_py/"
sz = 224
arch = resnet34
bs = 64

In [13]:
m = arch(True)

In [14]:
m = nn.Sequential(*children(m)[:-5], 
                  nn.Conv2d(64, 3, 3, padding=1), 
                  nn.AdaptiveAvgPool2d(1), Flatten(), 
                  nn.LogSoftmax())

In [15]:
m

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, ker

In [16]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs)

In [17]:
data

In [18]:
learn = ConvLearner.from_model_data(m, data)

In [19]:
learn.freeze_to(-4)

In [20]:
learn.fit(1e-4, 100)

epoch      trn_loss   val_loss   accuracy                   
    0      1.01698    0.983609   0.521358  
    1      0.979897   0.966781   0.525739                    
    2      0.967367   0.958548   0.518072                    
    3      0.959345   0.946086   0.53012                     
    4      0.952983   0.939766   0.533406                    
    5      0.948381   0.941381   0.538883                    
    6      0.942314   0.939315   0.522453                    
    7      0.932388   0.936038   0.531216                    
    8      0.929572   0.940321   0.536692                    
    9      0.927052   0.931942   0.547645                    
    10     0.928264   0.927146   0.542169                    
    11     0.926128   0.924984   0.543264                    
    12     0.924891   0.928244   0.54655                     
    13     0.929222   0.92778    0.547645                    
    14     0.91996    0.920892   0.549836                    
    15     0.912121   0.940

KeyboardInterrupt: 

In [19]:
learn.fit(1e-3, 25, cycle_len=1)

epoch      trn_loss   val_loss   accuracy                    
    0      0.929304   0.944784   0.538336  
    1      0.92884    0.938784   0.547036                    
    2      0.92454    0.938244   0.546493                    
    3      0.924133   0.941711   0.545405                    
 20%|██        | 23/114 [00:02<00:08, 10.81it/s, loss=0.923]

KeyboardInterrupt: 

## Class Activation Maps (CAM)

In [ ]:
class SaveFeatures():
    features=None
    def __init__(self, m): self.hook = m.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output): self.features = output
    def remove(self): self.hook.remove()

In [ ]:
x,y = next(iter(data.val_dl))

In [ ]:
x,y = x[None,1], y[None,1]
vx = Variable(x.cuda(), requires_grad=True)

In [ ]:
dx = data.val_ds.denorm(x)[0]
plt.imshow(dx);

In [ ]:
sfs = [SaveFeatures(o) for o in [m[-7], m[-6], m[-5], m[-4]]]

In [ ]:
%time py = m(Variable(x.cuda()))

In [ ]:
for o in sfs: o.remove()

In [ ]:
[o.features.size() for o in sfs]

In [ ]:
py = np.exp(to_np(py)[0]); py

In [ ]:
feat = np.maximum(0,to_np(sfs[3].features[0]))
feat.shape

In [ ]:
f2=np.dot(np.rollaxis(feat,0,3), py)
f2-=f2.min()
f2/=f2.max()
f2

In [ ]:
plt.imshow(dx)
plt.imshow(skimage.transform.resize(f2, dx.shape), alpha=0.5, cmap='hot');

## Model

In [ ]:
learn.unfreeze()
learn.bn_freeze(True)

In [ ]:
# 12 layer groups call for 12 lrs
lr=np.array([[1e-6]*4,[1e-4]*4,[1e-2]*4]).flatten()

In [ ]:
learn.fit(lr, 2, cycle_len=1)

In [ ]:
log_preds,y = learn.TTA()
preds = np.mean(np.exp(log_preds),0)
accuracy_np(preds,y)

In [ ]:
learn.fit(lr, 2, cycle_len=1)

In [ ]:
log_preds,y = learn.TTA()
preds = np.mean(np.exp(log_preds),0)
accuracy_np(preds,y)